In [ ]:
import numpy as np
import healpy as hp
from pathlib import Path

In [ ]:
output_dir = Path("/global/cfs/cdirs/cmb/www/pysm-data") / "dust_gnilc"

In [ ]:
datadir = output_dir / "raw"

In [ ]:
output_nside = 2048

In [ ]:
output_lmax = int(min(2.5 * output_nside, 8192 * 2))

## Large scales

In [ ]:
alm_log_pol_tens_large_scale = hp.read_alm(
    datadir
    / "gnilc_dust_largescale_template_logpoltens_alm_nside2048_lmax1024_complex64.fits.gz",
    hdu=(1, 2, 3),
)

In [ ]:
map_log_pol_tens_large_scale = hp.alm2map(
    alm_log_pol_tens_large_scale.astype(np.complex128), nside=output_nside
)

In [ ]:
map_log_pol_tens_large_scale

## Small scales modulation

In [ ]:
modulate_alm = {
    k: hp.read_alm(datadir / f"gnilc_dust_{k}_modulation_alms_lmax768.fits.gz").astype(
        np.complex128
    )
    for k in ["temperature", "polarization"]
}

In [ ]:
mod_i = hp.alm2map(modulate_alm["temperature"], output_nside)
mod_p = hp.alm2map(modulate_alm["polarization"], output_nside)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.figure(figsize=(20, 5))
r = {0: (3, 10), 1: (-0.2, 0.2)}
for i_m, m in enumerate([mod_i, mod_p]):
    hp.mollview(
        m,
        # title="Planck-GNILC 2058/2018 dust " + pol,
        title="",
        sub=121 + i_m,
        min=0,
        max=3,
        # cbar=False
    )

## Small scales

In [ ]:
cl_small_scale = hp.read_cl(
    datadir / "gnilc_dust_small_scales_logpoltens_cl_lmax16384.fits.gz"
)

In [ ]:
synalm_lmax = 8192 * 2  # it needs to be the same for all output nside
# synalm_lmax = 1024
np.random.seed(8192)

alm_log_pol_tens_small_scale = hp.synalm(
    list(cl_small_scale),
    lmax=synalm_lmax,
    new=True,
)

In [ ]:
alm_log_pol_tens_small_scale = [
    hp.almxfl(each, np.ones(output_lmax)) for each in alm_log_pol_tens_small_scale
]
map_log_pol_tens_small_scale = hp.alm2map(
    alm_log_pol_tens_small_scale, nside=output_nside
)

In [ ]:
map_log_pol_tens_small_scale[0] *= hp.alm2map(modulate_alm["temperature"], output_nside)
map_log_pol_tens_small_scale[1:] *= hp.alm2map(
    modulate_alm["polarization"], output_nside
)
assert np.isnan(map_log_pol_tens_small_scale).sum() == 0

In [ ]:
plt.figure(figsize=(20, 5))
for i_pol in range(3):
    r = 1.3 if i_pol == 0 else 0.1
    hp.mollview(
        map_log_pol_tens_small_scale[i_pol],
        # title="Planck-GNILC 2058/2018 dust " + pol,
        title="",
        sub=131 + i_pol,
        cbar=False,
        min=-r,
        max=r,
    )

In [ ]:
from pysm3.utils import log_pol_tens_to_map

In [ ]:
hp.gnomview(map_log_pol_tens_large_scale[0], xsize=1000, rot=80, title="")